## check GPU availabilty

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs Available: ", len(physical_devices))
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Number of GPUs Available:  1


## Paths

In [ ]:
CUR_DIR = '/content/' #this may be different runnig local
MAT_DIR= CUR_DIR+'mat/'
TRAIN_MAT_PATH = MAT_DIR + 'train/train/'
TEST_MAT_PATH = MAT_DIR + 'test/test/'
VOICE_DIR = CUR_DIR+'voice/'
TRAIN_VOICE_PATH = VOICE_DIR+'train/train/'
TEST_VOICE_PATH = VOICE_DIR+'test/test/'
DRIVE_PATH = CUR_DIR+'drive/MyDrive/Bachelor_Project/'
DRIVE2_PATH = CUR_DIR+'drive/MyDrive/Bachelor_Project_Part2/'
DRIVE3_PATH = CUR_DIR+'drive/MyDrive/Bachelor_Project_Part3/'
#TRAIN_VOICE_OLD_PATH = CUR_DIR+'train/train/'
#TEST_VOICE_OLD_PATH = CUR_DIR+'test/test/'
#TRAIN_IMG_PATH = CUR_DIR+'train/train-img/'
#TEST_IMG_PATH = CUR_DIR+'test/test-img/'
TRAIN_MAT_PATH = MAT_DIR + 'train/'
TEST_MAT_PATH = MAT_DIR + 'test/'
MOZILLA_PATH= CUR_DIR+'mozilla/'
MOZILLA_VALID_TRAIN = CUR_DIR+'mozilla/cv-valid-train/'
MOZILLA_VALID_DEV = CUR_DIR+'mozilla/cv-valid-dev/'
MOZILLA_VALID_TEST = CUR_DIR+'mozilla/cv-valid-test/'

In [ ]:
! mkdir $MAT_DIR
! mkdir $VOICE_DIR
! mkdir $TRAIN_MAT_PATH
! mkdir $TEST_MAT_PATH

## mount google drive

In [ ]:
from google.colab import drive
drive.mount(CUR_DIR+'drive/', force_remount=True)
#%cd '{CUR_DIR}drive/MyDrive/Bachelor_Project/'
#%ls

Mounted at /content/drive/


## import libraries



In [ ]:
# low import
import numpy as np
import pandas as pd
import gc

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from glob import glob
import librosa
import librosa.display
import soundfile as sf
import scipy.signal as signal
import gc
import shutil

## spoken-language-identification dataset


In [ ]:
! mkdir ~/.kaggle
! cp $DRIVE_PATH/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download toponowicz/spoken-language-identification

100% 14.9G/14.9G [02:51<00:00, 99.0MB/s]
100% 14.9G/14.9G [02:51<00:00, 93.5MB/s]


In [ ]:
%%capture
!unzip {CUR_DIR}spoken-language-identification.zip -d $VOICE_DIR

### create dataframe

In [ ]:
train_data_dir=np.array(glob(TRAIN_VOICE_PATH+"*"))
traindf = pd.DataFrame(train_data_dir,columns = ['file'])
format_len = len('flac')
traindf['ID']= traindf['file'].apply(lambda x: x.split('/')[-1][:-format_len]+'npy')
traindf

,file,ID
0,/content/voice/train/train/es_f_56ef0bde6c92a0...,es_f_56ef0bde6c92a0100da23cdd39a8a112.fragment...
1,/content/voice/train/train/en_f_c8835fb3edef24...,en_f_c8835fb3edef243176e1e9beef1f9681.fragment...
2,/content/voice/train/train/es_f_68e1dcf1b41edc...,es_f_68e1dcf1b41edcfa24b402f46ec2a65e.fragment...
3,/content/voice/train/train/en_m_6a5535d7512259...,en_m_6a5535d7512259e7bef76ff997750873.fragment...
4,/content/voice/train/train/de_f_d94712992f41e3...,de_f_d94712992f41e3d8d21f22274b3d8fd9.fragment...
...,...,...
73075,/content/voice/train/train/en_f_17211c49a9b296...,en_f_17211c49a9b2967b57cffaccf0d5b565.fragment...
73076,/content/voice/train/train/de_f_f856a0201f94b3...,de_f_f856a0201f94b3b57b61815ef8b715d2.fragment...
73077,/content/voice/train/train/es_f_bac9be13a058d1...,es_f_bac9be13a058d1c859ed5d2e3a0dc27b.fragment...
73078,/content/voice/train/train/de_f_5d2e7f30d69f2d...,de_f_5d2e7f30d69f2d1d86fd05f3bbe120c2.fragment...


In [ ]:
test_data_dir=np.array(glob(TEST_VOICE_PATH+"*"))
testdf = pd.DataFrame(test_data_dir,columns = ['file'])
format_len = len('flac')
testdf['ID']= testdf['file'].apply(lambda x: x.split('/')[-1][:-format_len]+'npy')
testdf

,file,ID
0,/content/voice/test/test/de_f_63f5b79c76cf5a1a...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...
1,/content/voice/test/test/de_f_63f5b79c76cf5a1a...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...
2,/content/voice/test/test/es_m_08111ce8d6a7ebc6...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...
3,/content/voice/test/test/en_f_67a0cba10d171b24...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...
4,/content/voice/test/test/en_f_67a0cba10d171b24...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...
...,...,...
535,/content/voice/test/test/en_m_b74b2bf2af570393...,en_m_b74b2bf2af570393cae91f4ed89cece7.fragment...
536,/content/voice/test/test/es_m_08111ce8d6a7ebc6...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...
537,/content/voice/test/test/de_f_63f5b79c76cf5a1a...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...
538,/content/voice/test/test/es_m_08111ce8d6a7ebc6...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...


In [ ]:
traindf['lang'] = traindf['ID'].apply(lambda x: x.split('_')[0])
testdf['lang'] = testdf['ID'].apply(lambda x: x.split('_')[0])

### create folders

In [ ]:
! mkdir $CUR_DIR/train
! mkdir $CUR_DIR/test

In [ ]:
! mv $TRAIN_VOICE_PATH $TRAIN_VOICE_OLD_PATH
! mv $TEST_VOICE_PATH $TEST_VOICE_OLD_PATH

In [ ]:
!mkdir $TRAIN_IMG_PATH
!mkdir $TEST_IMG_PATH

mkdir: missing operand
Try 'mkdir --help' for more information.
mkdir: missing operand
Try 'mkdir --help' for more information.


## Mozilla Dataset

In [ ]:
! kaggle datasets download mozillaorg/common-voice

In [ ]:
!mkdir mozilla

In [ ]:
%%capture
!unzip {CUR_DIR}common-voice.zip -d $MOZILLA_PATH

### remove invalid and other datasets

In [ ]:
!rm $MOZILLA_PATH/LICENSE.txt
!rm $MOZILLA_PATH/README.txt
!rm -r $MOZILLA_PATH/cv-other*
!rm -r $MOZILLA_PATH/cv-invalid*

In [ ]:
valid_train_df = pd.read_csv(MOZILLA_PATH+"cv-valid-train.csv")
valid_train_df

In [ ]:
valid_dev_df = pd.read_csv(MOZILLA_PATH+"cv-valid-dev.csv")
valid_dev_df

In [ ]:
valid_test_df = pd.read_csv(MOZILLA_PATH+"cv-valid-test.csv")
valid_test_df

In [ ]:
def process_mozilla_df(df, path_type):
    #df = df[pd.notnull(df['accent'])]
    df.dropna(subset=['accent'], inplace=True)
    df.rename(columns={'filename': 'file', 'accent': 'lang'}, inplace=True)
    df['ID'] = df['file'].str.split('/', expand=True)[1] + '.npy'
    df['file'] = path_type+df['file']

In [ ]:
process_mozilla_df(valid_train_df, MOZILLA_VALID_TRAIN)
process_mozilla_df(valid_dev_df, MOZILLA_VALID_DEV)
process_mozilla_df(valid_test_df, MOZILLA_VALID_TEST)

In [ ]:
valid_train_df

,file,text,up_votes,down_votes,age,gender,lang,duration,ID
5,/content/mozilla/cv-valid-train/cv-valid-train...,a shepherd may like to travel but he should ne...,1,0,twenties,female,us,NaN,sample-000005.mp3.jpg
8,/content/mozilla/cv-valid-train/cv-valid-train...,put jackie right on the staff,3,0,seventies,male,us,NaN,sample-000008.mp3.jpg
13,/content/mozilla/cv-valid-train/cv-valid-train...,but he had found a guide and didn't want to mi...,1,0,thirties,female,us,NaN,sample-000013.mp3.jpg
14,/content/mozilla/cv-valid-train/cv-valid-train...,as they began to decorate the hallway a silhou...,1,0,sixties,male,england,NaN,sample-000014.mp3.jpg
19,/content/mozilla/cv-valid-train/cv-valid-train...,then they got ahold of some dough and went goofy,1,0,fifties,male,australia,NaN,sample-000019.mp3.jpg
...,...,...,...,...,...,...,...,...,...
195766,/content/mozilla/cv-valid-train/cv-valid-train...,but before i go i want to tell you a little story,1,0,fourties,male,england,NaN,sample-195766.mp3.jpg
195767,/content/mozilla/cv-valid-train/cv-valid-train...,down below in the darkness were hundreds of pe...,3,0,thirties,female,us,NaN,sample-195767.mp3.jpg
195770,/content/mozilla/cv-valid-train/cv-valid-train...,he heard a muffled grating sound and saw the b...,4,0,twenties,male,england,NaN,sample-195770.mp3.jpg
195771,/content/mozilla/cv-valid-train/cv-valid-train...,the englishman said nothing,1,0,thirties,male,england,NaN,sample-195771.mp3.jpg


In [ ]:
valid_dev_df

,file,text,up_votes,down_votes,age,gender,lang,duration,ID
4,/content/mozilla/cv-valid-dev/cv-valid-dev/sam...,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN,sample-000004.mp3.jpg
5,/content/mozilla/cv-valid-dev/cv-valid-dev/sam...,but everything had changed,3,0,teens,male,us,NaN,sample-000005.mp3.jpg
8,/content/mozilla/cv-valid-dev/cv-valid-dev/sam...,the shop folks were taking down their shutters...,1,0,twenties,female,canada,NaN,sample-000008.mp3.jpg
9,/content/mozilla/cv-valid-dev/cv-valid-dev/sam...,the teacher thought that he'd taught himself a...,1,0,fifties,female,australia,NaN,sample-000009.mp3.jpg
13,/content/mozilla/cv-valid-dev/cv-valid-dev/sam...,in those days very few of the people had any i...,5,0,twenties,male,us,NaN,sample-000013.mp3.jpg
...,...,...,...,...,...,...,...,...,...
4053,/content/mozilla/cv-valid-dev/cv-valid-dev/sam...,if you start your emails with greetings let me...,1,0,sixties,female,canada,NaN,sample-004053.mp3.jpg
4055,/content/mozilla/cv-valid-dev/cv-valid-dev/sam...,the atmosphere was suffused with the sweet sce...,4,1,fourties,male,england,NaN,sample-004055.mp3.jpg
4067,/content/mozilla/cv-valid-dev/cv-valid-dev/sam...,they reached the center of a large plaza where...,3,0,fourties,female,us,NaN,sample-004067.mp3.jpg
4069,/content/mozilla/cv-valid-dev/cv-valid-dev/sam...,no sense messing up the streets,2,0,thirties,male,england,NaN,sample-004069.mp3.jpg


In [ ]:
valid_test_df

,file,text,up_votes,down_votes,age,gender,lang,duration,ID
3,/content/mozilla/cv-valid-test/cv-valid-test/s...,down below in the darkness were hundreds of pe...,4,0,twenties,male,us,NaN,sample-000003.mp3.jpg
5,/content/mozilla/cv-valid-test/cv-valid-test/s...,down below in the darkness were hundreds of pe...,4,1,twenties,male,us,NaN,sample-000005.mp3.jpg
8,/content/mozilla/cv-valid-test/cv-valid-test/s...,this was the strangest of all things that ever...,1,0,thirties,male,england,NaN,sample-000008.mp3.jpg
9,/content/mozilla/cv-valid-test/cv-valid-test/s...,it was glaringly hot not a cloud in the sky no...,3,0,fifties,male,us,NaN,sample-000009.mp3.jpg
14,/content/mozilla/cv-valid-test/cv-valid-test/s...,follow the instructions here,1,0,twenties,male,scotland,NaN,sample-000014.mp3.jpg
...,...,...,...,...,...,...,...,...,...
3971,/content/mozilla/cv-valid-test/cv-valid-test/s...,he's on my phone,1,0,teens,male,england,NaN,sample-003971.mp3.jpg
3975,/content/mozilla/cv-valid-test/cv-valid-test/s...,the planet's core is made of solid gold but ev...,1,0,twenties,male,us,NaN,sample-003975.mp3.jpg
3976,/content/mozilla/cv-valid-test/cv-valid-test/s...,we've got her located,1,0,thirties,male,england,NaN,sample-003976.mp3.jpg
3980,/content/mozilla/cv-valid-test/cv-valid-test/s...,i've got to see nicole right away,1,0,fifties,male,australia,NaN,sample-003980.mp3.jpg


## Managing dataset

### Load datasets

In [ ]:
first_half_df = pd.read_csv(DRIVE_PATH + "first_half.csv")

In [ ]:
first_half_df

,file,ID,lang
0,/content/voice/train/train/de_m_df90bffac026f7...,de_m_df90bffac026f705f4a99871d71684f1.fragment...,de
1,/content/voice/train/train/de_m_0345d503166fbc...,de_m_0345d503166fbcb125b6e10e2e804443.fragment...,de
2,/content/voice/train/train/en_m_d2f13c7f37ecec...,en_m_d2f13c7f37ecec67df0f46408c224bc1.fragment...,en
3,/content/voice/train/train/en_m_3a8f30bef6d7b6...,en_m_3a8f30bef6d7b6a1332629491eabcbf0.fragment...,en
4,/content/voice/train/train/en_f_7701e10392ce7b...,en_f_7701e10392ce7bcfba6d2fcca56fb8d9.fragment...,en
...,...,...,...
36535,/content/voice/train/train/de_f_26ddc4e756f612...,de_f_26ddc4e756f61233d74d42cc000c9dae.fragment...,de
36536,/content/voice/train/train/en_m_17b5d499519197...,en_m_17b5d4995191976232356345dfbb82cb.fragment...,en
36537,/content/voice/train/train/es_f_56ef0bde6c92a0...,es_f_56ef0bde6c92a0100da23cdd39a8a112.fragment...,es
36538,/content/voice/train/train/es_f_cc86b9208b120f...,es_f_cc86b9208b120f1f147a9246ab39f5e4.fragment...,es


In [ ]:
print(len(first_half_df[first_half_df['lang']  == 'en']))
print(len(first_half_df[first_half_df['lang']  == 'es']))
print(len(first_half_df[first_half_df['lang']  == 'de']))

12163
12200
12177


In [ ]:
print(first_half_df["ID"].iloc[1])

de_m_0345d503166fbcb125b6e10e2e804443.fragment25.speed4.npy


In [ ]:
%%time
format_len = len('flac')
np.vectorize(spectrogram_mat)(TRAIN_VOICE_PATH, first_half_df['ID'], TRAIN_MAT_PATH)

CPU times: user 36min 28s, sys: 26min 53s, total: 1h 3min 22s
Wall time: 41min


array([None, None, None, ..., None, None, None], dtype=object)

In [ ]:
! du -sh $TRAIN_MAT_PATH

7.6G	/content/mat/train/


In [ ]:
! rm -r $VOICE_DIR

In [ ]:
! mkdir {DRIVE3_PATH}first_half

In [ ]:
first_half_df.apply(lambda x: shutil.copy(TRAIN_MAT_PATH+x.ID, DRIVE3_PATH+"first_half/"+x.ID), axis=1)

0        /content/drive/MyDrive/Bachelor_Project_Part3/...
1        /content/drive/MyDrive/Bachelor_Project_Part3/...
2        /content/drive/MyDrive/Bachelor_Project_Part3/...
3        /content/drive/MyDrive/Bachelor_Project_Part3/...
4        /content/drive/MyDrive/Bachelor_Project_Part3/...
                               ...                        
36535    /content/drive/MyDrive/Bachelor_Project_Part3/...
36536    /content/drive/MyDrive/Bachelor_Project_Part3/...
36537    /content/drive/MyDrive/Bachelor_Project_Part3/...
36538    /content/drive/MyDrive/Bachelor_Project_Part3/...
36539    /content/drive/MyDrive/Bachelor_Project_Part3/...
Length: 36540, dtype: object

In [ ]:
first_half_df['lang'] = first_half_df['ID'].apply(lambda x: x.split('_')[0])

In [ ]:
first_half_df.to_csv(DRIVE_PATH+"first_half.csv")

In [ ]:
testdf

,file,ID
0,/content/voice/test/test/en_f_67a0cba10d171b24...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...
1,/content/voice/test/test/en_m_b74b2bf2af570393...,en_m_b74b2bf2af570393cae91f4ed89cece7.fragment...
2,/content/voice/test/test/es_f_50298ab71aaba850...,es_f_50298ab71aaba8508ebeef49d853df11.fragment...
3,/content/voice/test/test/de_f_63f5b79c76cf5a1a...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...
4,/content/voice/test/test/en_f_67a0cba10d171b24...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...
...,...,...
535,/content/voice/test/test/de_m_923551d571cc4373...,de_m_923551d571cc437382d0294dda2dd0aa.fragment...
536,/content/voice/test/test/en_m_b74b2bf2af570393...,en_m_b74b2bf2af570393cae91f4ed89cece7.fragment...
537,/content/voice/test/test/en_m_b74b2bf2af570393...,en_m_b74b2bf2af570393cae91f4ed89cece7.fragment...
538,/content/voice/test/test/de_m_923551d571cc4373...,de_m_923551d571cc437382d0294dda2dd0aa.fragment...


In [ ]:
testdf['lang'] = testdf['ID'].apply(lambda x: x.split('_')[0])

In [ ]:
testdf.to_csv(DRIVE_PATH+"testdf.csv")

In [ ]:
%%time
format_len = len('flac')
testdf = pd.DataFrame(test_data_dir,columns = ['file'])
testdf['ID']= testdf['file'].apply(lambda x: x.split('/')[-1][:-format_len]+'npy')
np.vectorize(spectrogram_mat)(TEST_VOICE_PATH, testdf['ID'], TEST_MAT_PATH)
testdf

### Move to label directory

In [ ]:
labels = ['en','es','de']
for x in labels:
    !mkdir $TRAIN_IMG_PATH/$x
    !mkdir $TEST_IMG_PATH/$x

In [ ]:
def move_to_label(name,lang, is_train=True):
    prefix= TRAIN_IMG_PATH
    if not is_train:
        prefix= TEST_IMG_PATH
    shutil.move(prefix+name,prefix+lang+"/"+name)
    #print(prefix+name, "to", prefix+lang+"/"+name)

In [ ]:
train_data_dir=np.array(glob(TRAIN_VOICE_PATH+"*"))
test_data_dir=np.array(glob(TEST_VOICE_PATH+"*"))

In [ ]:
train_data_dir

array(['/content/voice/train/train/de_m_df90bffac026f705f4a99871d71684f1.fragment28.noise3.flac',
       '/content/voice/train/train/en_f_9a47e3d56398b8108f59385aab8bbe87.fragment21.pitch5.flac',
       '/content/voice/train/train/de_f_2825fa225d6ca4800f0cf0504b76ca65.fragment14.speed6.flac',
       ...,
       '/content/voice/train/train/es_f_68e1dcf1b41edcfa24b402f46ec2a65e.fragment2.speed7.flac',
       '/content/voice/train/train/de_m_3aa01c55ba7ca85cad2977c848eb826f.fragment8.noise6.flac',
       '/content/voice/train/train/de_m_fc6bd6bb9d66a89bb8d8a8a7efa23e6b.fragment9.speed8.flac'],
      dtype='<U88')

In [ ]:
print(len(train_data_dir))

73080


In [ ]:
print(len(test_data_dir))

540


## Create spectogram

### ok spectrograms:

In [ ]:
def spectrogram_mat(address_prefix, file, save_prefix, format='flac'):
    clip, sample_rate = librosa.load(address_prefix+file[:-3]+format, sr=None)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    SP = librosa.power_to_db(S, ref=np.max)
    np.save(save_prefix+file, SP)

In [ ]:
def show_spectrogram_mat(filename, file, is_train=True):
    plt.interactive(False)
    ###clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    ###S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    ###SP = librosa.power_to_db(S, ref=np.max)
    #S = librosa.feature.mfcc(y=clip, sr=sample_rate)
    S= np.load(filename)
    #librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    librosa.display.specshow(S)
    prefix= TRAIN_IMG_PATH
    if not is_train:
        prefix= TEST_IMG_PATH
    name  = prefix + file[:-4] + ".jpg"
    #name2  = prefix + file[:-4] + "2.jpg"
    #np.save(name, S)
    #np.save(name2, SP)
    plt.savefig(name, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()
    fig.clf()
    plt.close(fig)
    plt.close('all')
    #del filename,name,clip,sample_rate,fig,ax,S

### ok generating

In [ ]:
%%time
format_len = len('flac')
traindf = pd.DataFrame(train_data_dir,columns = ['file'])
traindf['ID']= traindf['file'].apply(lambda x: x.split('/')[-1][:-format_len]+'npy')
np.vectorize(spectrogram_mat)(TRAIN_VOICE_PATH, traindf['ID'], TRAIN_MAT_PATH)
traindf

CPU times: user 47min 31s, sys: 32min 12s, total: 1h 19min 43s
Wall time: 49min 5s


,file,ID
0,/content/voice/train/train/de_m_df90bffac026f7...,de_m_df90bffac026f705f4a99871d71684f1.fragment...
1,/content/voice/train/train/en_f_9a47e3d56398b8...,en_f_9a47e3d56398b8108f59385aab8bbe87.fragment...
2,/content/voice/train/train/de_f_2825fa225d6ca4...,de_f_2825fa225d6ca4800f0cf0504b76ca65.fragment...
3,/content/voice/train/train/en_m_81995ee8a5e990...,en_m_81995ee8a5e990193b7858ec4b158e48.fragment...
4,/content/voice/train/train/en_m_1e512792ebab2b...,en_m_1e512792ebab2bd93b7aacac3d521390.fragment...
...,...,...
73075,/content/voice/train/train/de_f_8e8ed685b0cf9f...,de_f_8e8ed685b0cf9f5bbd13a04629ee64c5.fragment...
73076,/content/voice/train/train/es_f_47bd2e6178465c...,es_f_47bd2e6178465cd745c86c9db5ffe447.fragment...
73077,/content/voice/train/train/es_f_68e1dcf1b41edc...,es_f_68e1dcf1b41edcfa24b402f46ec2a65e.fragment...
73078,/content/voice/train/train/de_m_3aa01c55ba7ca8...,de_m_3aa01c55ba7ca85cad2977c848eb826f.fragment...


In [ ]:
%%time
format_len = len('flac')
testdf = pd.DataFrame(test_data_dir,columns = ['file'])
testdf['ID']= testdf['file'].apply(lambda x: x.split('/')[-1][:-format_len]+'npy')
np.vectorize(spectrogram_mat)(TEST_VOICE_PATH, testdf['ID'], TEST_MAT_PATH)
testdf

CPU times: user 29.8 s, sys: 21.6 s, total: 51.3 s
Wall time: 32.7 s


,file,ID
0,/content/voice/test/test/en_f_67a0cba10d171b24...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...
1,/content/voice/test/test/en_m_b74b2bf2af570393...,en_m_b74b2bf2af570393cae91f4ed89cece7.fragment...
2,/content/voice/test/test/es_f_50298ab71aaba850...,es_f_50298ab71aaba8508ebeef49d853df11.fragment...
3,/content/voice/test/test/de_f_63f5b79c76cf5a1a...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...
4,/content/voice/test/test/en_f_67a0cba10d171b24...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...
...,...,...
535,/content/voice/test/test/de_m_923551d571cc4373...,de_m_923551d571cc437382d0294dda2dd0aa.fragment...
536,/content/voice/test/test/en_m_b74b2bf2af570393...,en_m_b74b2bf2af570393cae91f4ed89cece7.fragment...
537,/content/voice/test/test/en_m_b74b2bf2af570393...,en_m_b74b2bf2af570393cae91f4ed89cece7.fragment...
538,/content/voice/test/test/de_m_923551d571cc4373...,de_m_923551d571cc437382d0294dda2dd0aa.fragment...


In [ ]:
! ls $TRAIN_MAT_PATH -1 | wc -l

73080


In [ ]:
! du -sh $TRAIN_MAT_PATH

16G	/content/mat/train/


In [ ]:
! mkdir {DRIVE3_PATH}"test/"

In [ ]:
show_spectrogram_mat('/content/train/train-img/de_f_26ddc4e756f61233d74d42cc000c9dae.fragment23.pitch8.flac2.npy','de_f_26ddc4e756f61233d74d42cc000c9dae.fragment23.pitch8.flac2.npy')

In [ ]:
traindf['lang'] = traindf['ID'].apply(lambda x: x.split('_')[0])

In [ ]:
move_to_label(traindf['ID'][0], traindf['lang'][0])

In [ ]:
np.vectorize(move_to_label, cache=True)(traindf['ID'], traindf['lang'])

### read data from drive

In [ ]:
### for first_half
first_half_df=pd.read_csv(DRIVE_PATH+'first_half.csv')
traindf = first_half_df

In [ ]:
first_half_df

,file,ID,lang
0,/content/voice/train/train/de_m_df90bffac026f7...,de_m_df90bffac026f705f4a99871d71684f1.fragment...,de
1,/content/voice/train/train/de_m_0345d503166fbc...,de_m_0345d503166fbcb125b6e10e2e804443.fragment...,de
2,/content/voice/train/train/en_m_d2f13c7f37ecec...,en_m_d2f13c7f37ecec67df0f46408c224bc1.fragment...,en
3,/content/voice/train/train/en_m_3a8f30bef6d7b6...,en_m_3a8f30bef6d7b6a1332629491eabcbf0.fragment...,en
4,/content/voice/train/train/en_f_7701e10392ce7b...,en_f_7701e10392ce7bcfba6d2fcca56fb8d9.fragment...,en
...,...,...,...
36535,/content/voice/train/train/de_f_26ddc4e756f612...,de_f_26ddc4e756f61233d74d42cc000c9dae.fragment...,de
36536,/content/voice/train/train/en_m_17b5d499519197...,en_m_17b5d4995191976232356345dfbb82cb.fragment...,en
36537,/content/voice/train/train/es_f_56ef0bde6c92a0...,es_f_56ef0bde6c92a0100da23cdd39a8a112.fragment...,es
36538,/content/voice/train/train/es_f_cc86b9208b120f...,es_f_cc86b9208b120f1f147a9246ab39f5e4.fragment...,es


### generate spectrogram

In [ ]:
def spectrogram_mat(address_prefix, file, save_prefix, format='flac'):
    clip, sample_rate = librosa.load(address_prefix+file[:-3]+format, sr=None)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    SP = librosa.power_to_db(S, ref=np.max)
    np.save(save_prefix+file, SP)

In [ ]:
%%time
format_len = len('flac')
np.vectorize(spectrogram_mat)(TRAIN_VOICE_PATH, traindf['ID'], TRAIN_MAT_PATH)
traindf

CPU times: user 54min 37s, sys: 40min 16s, total: 1h 34min 54s
Wall time: 58min 14s


,file,ID,lang
0,/content/voice/train/train/es_f_56ef0bde6c92a0...,es_f_56ef0bde6c92a0100da23cdd39a8a112.fragment...,es
1,/content/voice/train/train/en_f_c8835fb3edef24...,en_f_c8835fb3edef243176e1e9beef1f9681.fragment...,en
2,/content/voice/train/train/es_f_68e1dcf1b41edc...,es_f_68e1dcf1b41edcfa24b402f46ec2a65e.fragment...,es
3,/content/voice/train/train/en_m_6a5535d7512259...,en_m_6a5535d7512259e7bef76ff997750873.fragment...,en
4,/content/voice/train/train/de_f_d94712992f41e3...,de_f_d94712992f41e3d8d21f22274b3d8fd9.fragment...,de
...,...,...,...
73075,/content/voice/train/train/en_f_17211c49a9b296...,en_f_17211c49a9b2967b57cffaccf0d5b565.fragment...,en
73076,/content/voice/train/train/de_f_f856a0201f94b3...,de_f_f856a0201f94b3b57b61815ef8b715d2.fragment...,de
73077,/content/voice/train/train/es_f_bac9be13a058d1...,es_f_bac9be13a058d1c859ed5d2e3a0dc27b.fragment...,es
73078,/content/voice/train/train/de_f_5d2e7f30d69f2d...,de_f_5d2e7f30d69f2d1d86fd05f3bbe120c2.fragment...,de


In [ ]:
%%time
format_len = len('flac')
np.vectorize(spectrogram_mat)(TEST_VOICE_PATH, testdf['ID'], TEST_MAT_PATH)
testdf

CPU times: user 24 s, sys: 17.6 s, total: 41.6 s
Wall time: 25.3 s


,file,ID,lang
0,/content/voice/test/test/de_f_63f5b79c76cf5a1a...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
1,/content/voice/test/test/de_f_63f5b79c76cf5a1a...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
2,/content/voice/test/test/es_m_08111ce8d6a7ebc6...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
3,/content/voice/test/test/en_f_67a0cba10d171b24...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
4,/content/voice/test/test/en_f_67a0cba10d171b24...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
...,...,...,...
535,/content/voice/test/test/en_m_b74b2bf2af570393...,en_m_b74b2bf2af570393cae91f4ed89cece7.fragment...,en
536,/content/voice/test/test/es_m_08111ce8d6a7ebc6...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
537,/content/voice/test/test/de_f_63f5b79c76cf5a1a...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
538,/content/voice/test/test/es_m_08111ce8d6a7ebc6...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es


In [ ]:
print(traindf.iloc[0].ID)

es_f_56ef0bde6c92a0100da23cdd39a8a112.fragment13.speed3.npy


In [ ]:
gc.collect()

95711

In [ ]:
!rm {CUR_DIR}spoken-language-identification.zip

In [ ]:
!rm $VOICE_DIR -r

In [ ]:
traindf=pd.read_csv(DRIVE_PATH+'traindf.csv')

In [ ]:
! ls $TRAIN_MAT_PATH/first_half -1 | wc -l

ls: cannot access '/content/mat/train//first_half': No such file or directory
0


In [ ]:
! ls {DRIVE3_PATH}first_half -1 | wc -l

36540


In [ ]:
#%%time
#i= train_size #start i after train_size
#test_id = []
#for file in test_data_dir[i:i+test_size]:
#    name = file.split('/')[-1]
#    test_id.append(name+'.jpg')
#    create_spectrogram(file,name, is_train=False)

testdf = pd.DataFrame(test_data_dir[:test_size],columns = ['file'])
testdf['ID']= testdf['file'].apply(lambda x: x.split('/')[-1]+'.jpg')
#testdf.apply(lambda x: create_spectrogram(x.file, x.ID), axis=1)
np.vectorize(create_spectrogram)(testdf['file'], testdf['ID'], is_train=False)
testdf['lang'] = testdf['ID'].apply(lambda x: x.split('_')[0])
testdf

,file,ID,lang
0,/content/test/test/en_f_67a0cba10d171b24039a79...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
1,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
2,/content/test/test/es_m_08111ce8d6a7ebc6cd2c27...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
3,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
4,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
...,...,...,...
95,/content/test/test/en_f_67a0cba10d171b24039a79...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
96,/content/test/test/de_m_923551d571cc437382d029...,de_m_923551d571cc437382d0294dda2dd0aa.fragment...,de
97,/content/test/test/es_m_08111ce8d6a7ebc6cd2c27...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
98,/content/test/test/es_f_50298ab71aaba8508ebeef...,es_f_50298ab71aaba8508ebeef49d853df11.fragment...,es


In [ ]:
testdf

,file,ID,lang
0,/content/test/test/en_f_67a0cba10d171b24039a79...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
1,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
2,/content/test/test/es_m_08111ce8d6a7ebc6cd2c27...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
3,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
4,/content/test/test/de_f_63f5b79c76cf5a1a4bbd1c...,de_f_63f5b79c76cf5a1a4bbd1c40f54b166e.fragment...,de
...,...,...,...
95,/content/test/test/en_f_67a0cba10d171b24039a79...,en_f_67a0cba10d171b24039a79faa1d4d603.fragment...,en
96,/content/test/test/de_m_923551d571cc437382d029...,de_m_923551d571cc437382d0294dda2dd0aa.fragment...,de
97,/content/test/test/es_m_08111ce8d6a7ebc6cd2c27...,es_m_08111ce8d6a7ebc6cd2c27e62a3d98f0.fragment...,es
98,/content/test/test/es_f_50298ab71aaba8508ebeef...,es_f_50298ab71aaba8508ebeef49d853df11.fragment...,es


In [ ]:
file_paths = traindf['ID'].values
lang = np.array(traindf['lang'].values)

data = [np.load(file_path) for file_path in file_paths]
data = np.stack(data)

In [ ]:
%cd $CUR_DIR

In [ ]:
cur_shape = data.shape
data = data.reshape((cur_shape[0], cur_shape[1], cur_shape[2], 1))
data.shape

(10000, 128, 431, 1)

In [ ]:
! ls {DRIVE_PATH}/train -1 | wc -l

10000


In [ ]:
!pip install Keras-Preprocessing
from keras_preprocessing.image import ImageDataGenerator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.2) #.flow_from_


train_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=TRAIN_MAT_PATH+"train/",
    x_col="ID",
    y_col="lang",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

valid_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=TRAIN_MAT_PATH+"train/",
    x_col="ID",
    y_col="lang",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))



Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.10/dist-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 10000 invalid image filename(s) in x_col="ID". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 10000 invalid image filename(s) in x_col="ID". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
# npy
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.2) #.flow_from_


train_generator=datagen.flow(
    #dataframe=traindf,
    #directory=TRAIN_MAT_PATH+"train/",
    x=data,
    y=lang,
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    #class_mode="categorical",
    #target_size=(64,64)
    )

valid_generator=datagen.flow(
    #dataframe=traindf,
    #directory=TRAIN_MAT_PATH+"train/",
    x=data,
    y=lang,
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    #class_mode="categorical",
    #target_size=(64,64)
    )



TypeError: ignored

In [ ]:
data[0].shape

(128, 431, 1)

### new method

In [ ]:
gc.collect()

0

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
batch_size= 32
X_train, X_val, y_train, y_val = train_test_split(data, lang, test_size=0.2, shuffle=True, random_state=42)

# Reshape training data
X_train = X_train.reshape(X_train.shape[0], 128, 431, 1)
# Reshape validation data
X_val = X_val.reshape(X_val.shape[0], 128, 431, 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)


from keras.utils import to_categorical

y_train = to_categorical(y_train, 3)
y_val = to_categorical(y_val, 3)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)
#test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import keras

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(128, 431, 1)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 431, 32)      320       
                                                                 
 activation (Activation)     (None, 128, 431, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 429, 64)      18496     
                                                                 
 activation_1 (Activation)   (None, 126, 429, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 214, 64)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 214, 64)       0         
                                                        

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
model.fit(X_train, y_train, epochs=150, batch_size=batch_size, validation_data=(X_val, y_val))

Epoch 1/150
250/250 [==============================] - 90s 294ms/step - loss: 4.4136 - accuracy: 0.3298 - val_loss: 1.0988 - val_accuracy: 0.3275
Epoch 2/150
250/250 [==============================] - 68s 273ms/step - loss: 1.1139 - accuracy: 0.3396 - val_loss: 1.0990 - val_accuracy: 0.3275
Epoch 3/150
250/250 [==============================] - 71s 285ms/step - loss: 1.1062 - accuracy: 0.3345 - val_loss: 1.0954 - val_accuracy: 0.3640
Epoch 4/150
250/250 [==============================] - 259s 1s/step - loss: 1.0537 - accuracy: 0.4391 - val_loss: 0.9888 - val_accuracy: 0.5075
Epoch 5/150
250/250 [==============================] - 71s 284ms/step - loss: 0.9468 - accuracy: 0.5437 - val_loss: 0.8938 - val_accuracy: 0.6270
Epoch 6/150
250/250 [==============================] - 464s 2s/step - loss: 0.8191 - accuracy: 0.6366 - val_loss: 0.8211 - val_accuracy: 0.6300
Epoch 7/150
250/250 [==============================] - 70s 281ms/step - loss: 0.7175 - accuracy: 0.6888 - val_loss: 0.6062 - val

### old method

### save results

In [ ]:
model.save(CUR_DIR+'SampleCNN_150epoch_nostepsize.h5')

In [ ]:
model.save(DRIVE_PATH+"models/"+'SampleCNN_150epoch_nostepsize.h5')

In [ ]:
model.save(CUR_DIR+'model/Model_CNN1000.h5')
gc.collect()

0

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory=TEST_IMG_PATH,
    x_col="ID",
    y_col='lang',
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode='categorical',
    target_size=(64,64))
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

Found 100 validated image filenames belonging to 3 classes.


In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

<ipython-input-61-f904d4e6328a>:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_generator,


3/3 [==============================] - 1s 275ms/step


In [ ]:
predicted_class_indices.shape

(96,)

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (64, 64, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

image = load('/content/test/test-img/de_f_0809fd0642232f8c85b0b3d545dc2b5a.fragment18.pitch2.flac.jpg')
model.predict(image)

1/1 [==============================] - 0s 137ms/step


array([[0.00112203, 0.04741188, 0.951466  ]], dtype=float32)

In [ ]:
train_generator.class_indices

{'de': 0, 'en': 1, 'es': 2}

In [ ]:
prediction = model.predict(x=test_generator)

4/4 [==============================] - 1s 171ms/step


In [ ]:
print("accuracy on test:", np.sum(test_generator.classes == np.argmax(prediction, axis=-1)) / len(prediction) * 100)

accuracy on test: 43.0



## Paper's CNN without generator

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
batch_size= 32
X_train, X_val, y_train, y_val = train_test_split(data, lang, test_size=0.2, shuffle=True, random_state=42)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)


from keras.utils import to_categorical

y_train = to_categorical(y_train, 3)
y_val = to_categorical(y_val, 3)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)
#test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

## Paper's CNN

In [ ]:
from tensorflow import keras
import numpy as np
import os

class NumpyDataGenerator(keras.utils.Sequence):
    def __init__(self, data_folder, ids, labels, batch_size, shuffle=True, seed=None, rescale=None, subset='training'):
        self.data_folder = data_folder
        ## changed:
        #self.ids = ids
        self.file_list = ids
        #self.labels = labels
        self.file_labels = labels

        self.batch_size = batch_size
        #self.validation_split = validation_split
        self.shuffle = shuffle
        self.seed = seed
        self.rescale = rescale
        self.subset = subset

        #self._split_data()

        if self.shuffle:
            self._shuffle_data()

    #def _split_data(self):
    #    split_index = int(self.validation_split * len(self.ids))
    #    if self.subset == 'training':
    #        self.file_list = self.ids[split_index:]
    #        self.file_labels = self.labels[split_index:]
    #    else:
    #        self.file_list = self.ids[:split_index]
    #        self.file_labels = self.labels[:split_index]

    def _shuffle_data(self):
        if self.seed is not None:
            np.random.seed(self.seed)
        indices = np.random.permutation(len(self.file_list))
        self.file_list = self.file_list[indices]
        self.file_labels = self.file_labels[indices]

    def __len__(self):
        return len(self.file_list) // self.batch_size

    def __getitem__(self, idx):
        r = (idx + 1) * self.batch_size
        if r > len(self.file_list):
            r = len(self.file_list)
        batch_files = self.file_list[idx * self.batch_size: r]
        batch_labels = self.file_labels[idx * self.batch_size: r]

        batch_data = np.empty((len(batch_files), *(128, 431)))
        for i, filename in enumerate(batch_files):
            batch_data[i,] = np.load(os.path.join(self.data_folder, filename))

        return batch_data, batch_labels

    def on_epoch_end(self):
        if self.shuffle:
            self._shuffle_data()


In [ ]:
X_trainid = np.array(traindf['ID'].values)
y_train = np.array(traindf['lang'].values)

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_train = to_categorical(y_train, 3)

In [ ]:
traindf

,file,ID,lang
0,/content/voice/train/train/es_f_56ef0bde6c92a0...,es_f_56ef0bde6c92a0100da23cdd39a8a112.fragment...,es
1,/content/voice/train/train/en_f_c8835fb3edef24...,en_f_c8835fb3edef243176e1e9beef1f9681.fragment...,en
2,/content/voice/train/train/es_f_68e1dcf1b41edc...,es_f_68e1dcf1b41edcfa24b402f46ec2a65e.fragment...,es
3,/content/voice/train/train/en_m_6a5535d7512259...,en_m_6a5535d7512259e7bef76ff997750873.fragment...,en
4,/content/voice/train/train/de_f_d94712992f41e3...,de_f_d94712992f41e3d8d21f22274b3d8fd9.fragment...,de
...,...,...,...
73075,/content/voice/train/train/en_f_17211c49a9b296...,en_f_17211c49a9b2967b57cffaccf0d5b565.fragment...,en
73076,/content/voice/train/train/de_f_f856a0201f94b3...,de_f_f856a0201f94b3b57b61815ef8b715d2.fragment...,de
73077,/content/voice/train/train/es_f_bac9be13a058d1...,es_f_bac9be13a058d1c859ed5d2e3a0dc27b.fragment...,es
73078,/content/voice/train/train/de_f_5d2e7f30d69f2d...,de_f_5d2e7f30d69f2d1d86fd05f3bbe120c2.fragment...,de


In [ ]:
np.random.seed(42)
shuffle_indices = np.random.permutation(len(X_trainid))
X_trainid = X_trainid[shuffle_indices]
y_train = y_train[shuffle_indices]

In [ ]:
import copy
def split_data(X_id, y_label, validation_split):
    split_index = int(validation_split * len(X_id))
    X_train_id = copy.deepcopy(X_id[split_index:])
    y_train_label = copy.deepcopy(y_label[split_index:])
    X_valid_id = copy.deepcopy(X_id[:split_index])
    y_valid_label = copy.deepcopy(y_label[:split_index])
    return X_train_id, y_train_label, X_valid_id, y_valid_label

In [ ]:
X_train_id, y_train_label, X_valid_id, y_valid_label = split_data(X_trainid, y_train, 0.2)

In [ ]:
print(len(X_train_id), len(y_train_label), len(X_valid_id), len(y_valid_label))

58464 58464 14616 14616


In [ ]:
print(X_trainid)

['es_f_bac9be13a058d1c859ed5d2e3a0dc27b.fragment16.speed3.npy'
 'de_m_502293f229751563d1768dd2367a6b29.fragment19.speed7.npy'
 'es_m_d7128fe46e362acdb2271a154e3206ae.fragment6.noise5.npy' ...
 'en_f_c8835fb3edef243176e1e9beef1f9681.fragment22.noise12.npy'
 'de_f_404792fda5c66b95903adafd5a5f5d10.fragment15.pitch7.npy'
 'en_f_386ee651f6f1539ff5622c55e234e5a4.fragment6.noise5.npy']


In [ ]:
print(y_train)

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [ ]:
print(X_trainid[3], y_train[3])

en_m_d630900a88e2681c9d5d1026d7025d67.fragment5.pitch6.npy [0. 1. 0.]


In [ ]:
print(str(y_train[0]))

[0. 0. 1.]


In [ ]:
data_folder = TRAIN_MAT_PATH
batch_size = 32
validation_split = 0.2
rescale_factor = 1.0 / 255.0

params = {
    'data_folder': data_folder,
    ##'ids' : X_trainid,
    ##'labels': y_train,
    'batch_size': batch_size,
    #'validation_split': validation_split,
    'shuffle': True,
    'seed': 42,
    'rescale': rescale_factor,
}

train_generator = NumpyDataGenerator(subset='training', ids= X_train_id, labels= y_train_label, **params)
valid_generator = NumpyDataGenerator(subset='validation', ids= X_valid_id, labels= y_valid_label, **params)

In [ ]:
file_paths = traindf['ID'].values
y_train = np.array(traindf['lang'].values)

X_train = [np.load(MAT_DIR+"train/train/"+file_path) for file_path in file_paths]
X_train = np.stack(X_train)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_valid_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_valid_datagen.flow(X_train, y_train,
                                           batch_size=32,
                                           subset='training',
                                           seed=42,
                                           shuffle=True)

val_generator = train_valid_datagen.flow(X_train, y_train,
                                         batch_size=32,
                                         subset='validation',
                                         seed=42,
                                         shuffle=True)

#test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
#from keras import regularizers, optimizers
import keras

model = Sequential()
#first block:
model.add(Conv2D(32, (7, 7), padding='valid', input_shape=(128, 431, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
#second block:
model.add(Conv2D(64, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
#third block:
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
#fourth block:
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
#fifth block:
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
#flatten layer:
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
#model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
#              loss=keras.losses.CategoricalCrossentropy(),
#              metrics=["accuracy"])
#model.summary()

In [ ]:
#model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 122, 425, 32)      1600      
                                                                 
 batch_normalization (BatchN  (None, 122, 425, 32)     128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 122, 425, 32)      0         
                                                                 
 dropout (Dropout)           (None, 122, 425, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 60, 212, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 212, 64)       5

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)

Epoch 1/60
1827/1827 [==============================] - 320s 168ms/step - loss: 0.6341 - accuracy: 0.7370 - val_loss: 5.6170 - val_accuracy: 0.3324
Epoch 2/60
 157/1827 [=>............................] - ETA: 3:59 - loss: 0.2371 - accuracy: 0.9076

KeyboardInterrupt: ignored

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // 32,
    epochs=60,
    validation_data=val_generator,
    validation_steps= val_generator.n // 32
)

In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import keras

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(128, 431, 1)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=60,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)